# Data Mining: Harry Potter Sorting Hat
Studente: Aloisio Chiara Ludovica
Matricola: 239648
A/A: 2024/2025

---


# Setup
Come prima cosa, assicuriamoci che il seguente notebook funzioni sia con python 2 che 3, importiamo alcuni moduli di utilità e prepariamo le funzioni per salvare le immagini

In [ ]:
# Python ≥3.5 is required
import sys

from sklearn.metrics import accuracy_score

assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

#imports pandas
import pandas as pd
from IPython.display import Image
import os
#import warnings and Repress Warnings
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
warnings.filterwarnings("ignore", message="numpy.ndarray size changed")
warnings.filterwarnings('ignore')

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid") #White Grid
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = ".."
CHAPTER_ID= "Harry-Potter"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=400):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

def save_figure(figure, fig_id, tight_layout=False, fig_extension="png", resolution=400):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    figure.savefig(path, format=fig_extension, dpi=resolution)

#palette colori
# Tavola colori personalizzata per ogni casata
hue_palette = {
    0: '#9C2A2A',  # Rosso Grifondoro
    1: '#F1C232',  # Giallo Hufflepuff
    2: '#000080',  # Blu Corvonero
    3: '#1A7F3C'   # Verde Serpeverde
}


# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")
import warnings
warnings.filterwarnings('ignore')

# Hrule--> "====="
hrule = lambda x : "="*x
Hrule = lambda x,y: "="*(x//2)+y+"="*(x//2)

---

# Motivazione dello studio
Harry Potter è una serie di romanzi fantasy scritta da J.K. Rowling che segue le avventure di un giovane mago, Harry, mentre frequenta la Scuola di Magia e Stregoneria di Hogwarts e affronta il potente mago oscuro Lord Voldemort.

In [ ]:
from IPython.display import Image
Image(url="https://cdn.wallpapersafari.com/39/28/s9IzxG.jpg", width=800)


Come sceglie le casate il Cappello Parlante?
Il Cappello Parlante è un cappello magico che smista gli studenti nelle quattro casate di Hogwarts: Grifondoro, Serpeverde, Corvonero e Tassorosso. Durante la Cerimonia di Smistamento, il cappello viene posto sulla testa dello studente e legge nella sua mente desideri, qualità e potenziale. In base a ciò che trova, decide la casata più adatta. Si basa principalmete su alcune caratteristiche più rilevanti quali, ad esempio l'eredità genetica dei genitori, il coraggio, l'inelligenza, la lealtà, l'ambizione, la conoscenza delle arti oscure, abilità el giocare a quidditch, abilità nei duelli e creatività.

In [ ]:
from IPython.display import Image
Image(url="https://th.bing.com/th/id/OIP.gfkCacIGJu_4w9Q08F8G9gHaFu?rs=1&pid=ImgDetMain", width=800)


L'obiettivo del seguente studio sarà, dunque, dopo aver studiato il dataset, quello di predire la casata di uno studente della scuola di magia e stregoneria di Hogwarts attraverso modelli di multi classificazione.

---

# Pre processing
In questa prima parta andiamo a caricare i dati che verranno sottoposti a delle prime analisi di tipo strutturale che necesariamente porteranno all'individuazione di alcune trasformazioni necessarie per il proseguo delle analisi.

In [ ]:
df = pd.read_csv("../harry_potter_students.csv")
pd.set_option('display.max_columns', None)
print(df.columns)

df.head
print(df.shape)

Vediamo le informazioni sul dataset

In [ ]:
df.info()

Il comando `df.info()` mostra che il DataFrame ha 20.000 righe e 10 colonne. Otto colonne contengono dati numerici (`float64`) come Bravery, Intelligence, ecc., mentre due sono di tipo `object` (probabilmente stringhe), come Blood Status e House.
Tutte le colonne hanno valori mancanti, quindi sarà necessario gestire i `NaN`. La memoria usata è circa **1.5 MB**.
Iniziamo dunque i primi lavori di trasformazione del dataset, gestendo i valori NaN e eventuali duplicati al fine di aiutare la valutazione dei vari algoritmi.



Iniziamo eliminand i duplicati

In [ ]:
df.drop_duplicates(keep='first', inplace=True)
print(df.shape)

Eliminiamo le colonne che no ci servono, nel nostro caso id:

In [ ]:
#Scrivere codice
df.drop('id', axis=1, inplace=True)

df.head()

Andiamo ora a gestire i valori NaN:



In [ ]:
df.isnull().sum()

Decido di voler predire i NaN usando il modello KNN imputer, ma prima devo effettuare alcune conversioni, decidendo di effettuare label encoding sugli attributi "Blood Status", "name", "surname" e "House":


In [ ]:
print(df['House'].unique())


In [ ]:
#from sklearn.preprocessing import LabelEncoder

# Colonne da codificare
columns_to_encode = ['name', 'surname', 'House', 'Blood Status']

# Dizionario per salvare gli encoder
label_encoders = {}

for col in columns_to_encode:
    # Trasforma in maiuscolo solo i valori non nulli
    df[col] = df[col].apply(lambda x: x.upper() if pd.notnull(x) else x)

    # Crea un encoder e applicalo solo ai valori non nulli
    le = LabelEncoder()
    non_null_mask = df[col].notnull()
    df.loc[non_null_mask, col] = le.fit_transform(df.loc[non_null_mask, col])

    # Salva l'encoder
    label_encoders[col] = le

# Ora df ha le colonne codificate, e i NaN sono ancora NaN
df.head(60)



In [ ]:
encoder = label_encoders['House']
encoder1 = label_encoders['Blood Status']
encoder2 = label_encoders['name']
encoder3 = label_encoders['surname']

#visulizziamo il label encoding appena effettuato
for i, label in enumerate(encoder.classes_):
    print(f"{i} → {label}")

print("\n")
print("\n")
for i, label in enumerate(encoder1.classes_):
    print(f"{i} → {label}")

print("\n")
print("\n")
for i, label in enumerate(encoder2.classes_):
    print(f"{i} → {label}")

print("\n")
print("\n")
for i, label in enumerate(encoder3.classes_):
    print(f"{i} → {label}")

In [ ]:
from sklearn.impute import KNNImputer

# Colonne categoriali da convertire in interi dopo l'imputazione
categorical_columns = ['name', 'surname', 'House', 'Blood Status']

# Applica KNNImputer
knn_imputer = KNNImputer(n_neighbors=2)
df = pd.DataFrame(knn_imputer.fit_transform(df), columns=df.columns)

# Arrotonda e converte in interi solo le colonne categoriali
for col in categorical_columns:
    df[col] = df[col].round().astype(int)

# Visualizza il risultato
print(df)


Ho sovrascritto il dataset con i miei valori aggiornati. Infatti:

In [ ]:
df.isnull().sum()

ora effettivamente non ho più valori null.

---

# Visualizazion


In [ ]:
import seaborn as sns

# Pairplot del dataset (mostra scatter plot e distribuzioni per ogni variabile)
sns.pairplot(df)

# Mostrare il grafico
plt.show()


In [ ]:
#Analizzare la relazione tra abilità e casata
import seaborn as sns
import matplotlib.pyplot as plt

# Pairplot delle abilità colorato per House
sns.pairplot(
    df,
    vars=['Bravery', 'Intelligence', 'Loyalty', 'Ambition'],
    hue='House',
    palette=hue_palette,
    corner=True
)
plt.suptitle("Bravery vs Intelligence vs Loyalty vs Ambition", y=1.02)
plt.show()


In [ ]:
#Analizzare la relazione tra capacità e casate
sns.pairplot(
    df,
    vars=['Dark Arts Knowledge', 'Dueling Skills', 'Quidditch Skills'],
    hue='House',
    palette=hue_palette,
    corner=True
)
plt.suptitle("Dark Arts / Dueling / Quidditch vs House", y=1.02)
plt.show()


In [ ]:
sns.pairplot(
    df,
    vars=['Creativity', 'Intelligence', 'age'],
    hue='House',
    palette=hue_palette,
    corner=True
)
plt.suptitle("Creativity, Intelligence e Age in relazione alla House", y=1.02)
plt.show()


In [ ]:
selected_features = [
    'Bravery', 'Intelligence', 'Loyalty', 'Ambition',
    'Dark Arts Knowledge', 'Dueling Skills',
    'Quidditch Skills', 'Creativity', 'age'
]

sns.pairplot(df, vars=selected_features, hue='House', palette=hue_palette, corner=True, plot_kws={'alpha': 0.6, 's': 40})
plt.suptitle("Panoramica abilità e età per House", y=1.02)
plt.show()


In [ ]:
import seaborn as sns

# Calcolare la matrice di correlazione
corr_matrix = df.corr()

# Creare una heatmap della matrice di correlazione
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)

# Mostrare il grafico
plt.title('Correlation Heatmap')
plt.show()

---

# Transformation

Dopo aver determinato gli attributi da usare nel processo di regressione, si provvede a preparare i dati da passare agli algoritmi di machine learning. In particolare le trasformazioni che seguiranno non rientrano nel data cleaning, che invece è stato effettuato a monte, quanto principalmente la discretizzazione, l'omogeneizzazione e la codifica dei dati.

In [ ]:
df.name.unique()

df['name'].value_counts().head()

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler_std = StandardScaler()
df_scaled_std = pd.DataFrame(scaler_std.fit_transform(df), columns=df.columns)

scaler_minmax = MinMaxScaler()
df_scaled_minmax = pd.DataFrame(scaler_minmax.fit_transform(df), columns=df.columns)


In [ ]:
# Confronto distribuzioni dopo scaling
plt.figure(figsize=(12, 6))
sns.kdeplot(df_scaled_std["Bravery"], label="Standard Scaled")
sns.kdeplot(df_scaled_minmax["Bravery"], label="Min-Max Scaled")
plt.title("Distribuzione della feature 'Bravery' dopo scaling")
plt.legend()
plt.show()


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)  # Conserva il 95% della varianza
df_pca = pd.DataFrame(pca.fit_transform(df_scaled_std))


In [ ]:
#Visualizzazione PCA
plt.figure(figsize=(8, 6))
sns.scatterplot(x=df_pca[0], y=df_pca[1])
plt.title("PCA - Prima e Seconda Componente")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()


In [ ]:
df_engineered = df.copy()
df_engineered["Courage"] = df_engineered["Bravery"] + df_engineered["Loyalty"]
df_engineered["Magic Index"] = df_engineered[["Dark Arts Knowledge", "Dueling Skills", "Quidditch Skills"]].mean(axis=1)


In [ ]:
#visualizzazione feature ingegnerizzate
plt.figure(figsize=(10, 5))
sns.histplot(df_engineered["Courage"], bins=30, kde=True, color='orange')
plt.title("Distribuzione della nuova feature 'Courage'")
plt.show()

plt.figure(figsize=(10, 5))
sns.histplot(df_engineered["Magic Index"], bins=30, kde=True, color='purple')
plt.title("Distribuzione della nuova feature 'Magic Index'")
plt.show()

In [ ]:

df_log_transformed = df.copy()
for col in df_log_transformed.columns:
    df_log_transformed[col] = np.log1p(df_log_transformed[col])  # log(1 + x)


In [ ]:
#Confronto distribuzione originale vs log-transformed
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
sns.histplot(df["Dark Arts Knowledge"], bins=30, kde=True, ax=axes[0])
axes[0].set_title("Originale - Dark Arts Knowledge")

sns.histplot(df_log_transformed["Dark Arts Knowledge"], bins=30, kde=True, ax=axes[1], color='green')
axes[1].set_title("Log Trasformata - Dark Arts Knowledge")
plt.show()


In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

if "age" in df.columns:
    discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
    df_binned = df.copy()
    df_binned["age_group"] = discretizer.fit_transform(df_binned[["age"]])


In [ ]:
#Visualizzazione gruppi di età
sns.countplot(x="age_group", data=df_binned)
plt.title("Distribuzione delle età binned")
plt.xlabel("Gruppo di Età (0 = giovane, 2 = adulto)")
plt.ylabel("Conteggio")
plt.show()

Prima di iniziare ad addestrare gli algoritmi, andiamo ad effettuare il drop della colonna che vogliamo andare a predirre, ovvero 'House'

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize

# Feature/target split
X = df.drop("House", axis=1)
y = df["House"]

# Binarizza il target per AUC multiclass
classes = y.unique()
y_bin = label_binarize(y, classes=classes)

# Split
X_train, X_test, y_train, y_test, y_train_bin, y_test_bin = train_test_split(
    X, y, y_bin, test_size=0.2, random_state=42)



# Ensamble

In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.metrics import roc_auc_score

## RandomForest

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_prob_rf = rf.predict_proba(X_test)
auc_rf = roc_auc_score(y_test_bin, y_prob_rf, multi_class="ovr", average="macro")
print("Random Forest AUC:", auc_rf)

## AdaBoost

In [ ]:
ada = AdaBoostClassifier(n_estimators=100, random_state=42)
ada.fit(X_train, y_train)
y_prob_ada = ada.predict_proba(X_test)
auc_ada = roc_auc_score(y_test_bin, y_prob_ada, multi_class="ovr", average="macro")
print("AdaBoost AUC:", auc_ada)

## Bagging

In [ ]:
bag = BaggingClassifier(n_estimators=100, random_state=42)
bag.fit(X_train, y_train)
y_prob_bag = bag.predict_proba(X_test)
auc_bag = roc_auc_score(y_test_bin, y_prob_bag, multi_class="ovr", average="macro")
print("Bagging AUC:", auc_bag)


## Algoritmi a confronto:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

auc_scores = {
    "Random Forest": auc_rf,
    "AdaBoost": auc_ada,
    "Bagging": auc_bag
}

plt.figure(figsize=(8, 4))
sns.barplot(x=list(auc_scores.keys()), y=list(auc_scores.values()), palette="viridis")
plt.title("Confronto AUC tra modelli ensemble")
plt.ylabel("Macro AUC")
plt.ylim(0, 1)
plt.show()


In [ ]:
from sklearn.metrics import accuracy_score

# Accuracy
acc_rf = accuracy_score(y_test, rf.predict(X_test))
acc_ada = accuracy_score(y_test, ada.predict(X_test))
acc_bag = accuracy_score(y_test, bag.predict(X_test))

print(f"Accuracy Random Forest: {acc_rf:.3f}")
print(f"Accuracy AdaBoost: {acc_ada:.3f}")
print(f"Accuracy Bagging: {acc_bag:.3f}")


In [ ]:
from sklearn.model_selection import GridSearchCV

# Random Forest
param_grid_rf = {
    "n_estimators": [50, 100, 200],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5]
}
gs_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=5, scoring="accuracy", n_jobs=-1)
gs_rf.fit(X_train, y_train)
print("Best RF:", gs_rf.best_params_, "Acc:", gs_rf.best_score_)

# AdaBoost
param_grid_ada = {
    "n_estimators": [50, 100, 200],
    "learning_rate": [0.5, 1.0, 1.5]
}
gs_ada = GridSearchCV(AdaBoostClassifier(random_state=42), param_grid_ada, cv=5, scoring="accuracy", n_jobs=-1)
gs_ada.fit(X_train, y_train)
print("Best AdaBoost:", gs_ada.best_params_, "Acc:", gs_ada.best_score_)

# Bagging
param_grid_bag = {
    "n_estimators": [50, 100, 200],
    "max_samples": [0.5, 0.7, 1.0]
}
gs_bag = GridSearchCV(BaggingClassifier(random_state=42), param_grid_bag, cv=5, scoring="accuracy", n_jobs=-1)
gs_bag.fit(X_train, y_train)
print("Best Bagging:", gs_bag.best_params_, "Acc:", gs_bag.best_score_)


In [ ]:
acc_scores = {
    "Random Forest": acc_rf,
    "AdaBoost": acc_ada,
    "Bagging": acc_bag
}

plt.figure(figsize=(8, 4))
sns.barplot(x=list(acc_scores.keys()), y=list(acc_scores.values()), palette="mako")
plt.title("Confronto Accuracy tra modelli")
plt.ylabel("Accuracy")
plt.ylim(0, 1)
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Lista di modelli e nomi
models = [
    ("Random Forest", rf),
    ("AdaBoost", ada),
    ("Bagging", bag)
]

plt.figure(figsize=(15, 4))

for i, (name, model) in enumerate(models):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred, labels=classes)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    plt.subplot(1, 3, i + 1)
    disp.plot(ax=plt.gca(), cmap="Blues", colorbar=False)
    plt.title(name)

plt.tight_layout()
plt.show()


Come possiamo notare da queste confusion matrix, questi algoritmi non lavorano molto bene. Ciò accade perchè non è stato eseguito il tuning degli iperparameter. Procediamo dunque a fare ciò e facciamo lavorare nuovamente gli algoritmi

In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (confusion_matrix, ConfusionMatrixDisplay,
                             precision_score, recall_score, f1_score, accuracy_score, classification_report,
                             roc_curve, auc, RocCurveDisplay)
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import label_binarize
import numpy as np

# Dizionari dei parametri per ciascun classificatore
param_grids = {
    "Random Forest": {
        "n_estimators": [100, 200],
        "max_depth": [None, 10],
        "max_features": ["sqrt", "log2"]
    },
    "AdaBoost": {
        "n_estimators": [50, 100],
        "learning_rate": [0.5, 1.0]
    },
    "Bagging": {
        "n_estimators": [10, 50],
        "max_samples": [0.5, 1.0]
    }
}

# Classificatori
classifiers = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "Bagging": BaggingClassifier(random_state=42)
}

# Per salvare i migliori modelli e le metriche
best_models = {}
metrics_summary = []

# Grid Search e valutazione
for name, clf in classifiers.items():
    print(f"🔍 Grid Search per {name}...")
    grid = GridSearchCV(clf, param_grids[name], cv=5, scoring='accuracy', n_jobs=-1)
    grid.fit(X_train, y_train)
    best_models[name] = grid.best_estimator_

    print(f"✅ Migliori parametri per {name}: {grid.best_params_}")
    print(f"🎯 Accuracy media cross-val: {grid.best_score_:.4f}")

    # Predizione e metriche
    y_pred = grid.best_estimator_.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='macro')
    rec = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    # Confusion Matrix
    print(f"📊 Confusion Matrix per {name}:")
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f"{name} - Confusion Matrix")
    plt.show()

    # Report
    print(f"📋 Classification Report per {name}:")
    print(classification_report(y_test, y_pred))

    # ROC Curve e AUC
    print(f"📈 ROC Curve e AUC per {name}:")
    if len(np.unique(y_test)) == 2:
        # Caso binario
        y_score = grid.best_estimator_.predict_proba(X_test)[:, 1]
        fpr, tpr, _ = roc_curve(y_test, y_score)
        roc_auc = auc(fpr, tpr)

        plt.figure()
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f"{name} - ROC Curve")
        plt.legend(loc="lower right")
        plt.show()
    else:
        # Caso multi-classe
        y_test_bin = label_binarize(y_test, classes=np.unique(y_test))
        y_score = grid.best_estimator_.predict_proba(X_test)
        n_classes = y_test_bin.shape[1]

        fpr = dict()
        tpr = dict()
        roc_auc = dict()

        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])

        # Media macro dell'AUC
        all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
        mean_tpr = np.zeros_like(all_fpr)
        for i in range(n_classes):
            mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

        mean_tpr /= n_classes
        macro_auc = auc(all_fpr, mean_tpr)

        plt.figure()
        for i in range(n_classes):
            plt.plot(fpr[i], tpr[i], lw=2, label=f'Classe {i} (AUC = {roc_auc[i]:.2f})')
        plt.plot([0, 1], [0, 1], 'k--', lw=2)
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f"{name} - ROC Curve Multiclasse (macro AUC = {macro_auc:.2f})")
        plt.legend(loc="lower right")
        plt.show()

    # Salvataggio metriche
    metrics_summary.append({
        "Model": name,
        "Accuracy": acc,
        "Precision (macro)": prec,
        "Recall (macro)": rec,
        "F1-score (macro)": f1
    })

# Mostriamo la classifica finale
df_metrics = pd.DataFrame(metrics_summary)
df_sorted = df_metrics.sort_values(by="F1-score (macro)", ascending=False).reset_index(drop=True)

print("\n🏆 Classifica finale dei modelli (ordinata per F1-score macro):")
display(df_sorted)


# aggiungere confronto degli algoritmi prima e dopo del preprocessing

---

# Regression